Conversations and Tools

Stores and automatically handles conversations

In [ ]:
from langgraph.graph import MessagesState

class MyMessagesState(MessagesState):
    # Inherits from MessagesState, which automatically handles adding messages
    pass

In [ ]:
#llm
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")  # Just an example model name

Adding Tools

In [ ]:
def multiply(a: int, b: int) -> int:
    return a * b
    
llm_with_tools = llm.bind_tools([multiply])

Creating a Node for ai processing

1 -Reads the entire conversation from state["messages"].

2 -Invokes llm_with_tools to generate a response, considering all past messages.

3 -Returns that new message so it’s appended to the state.

In [ ]:
def tool_calling_llm(state: MyMessagesState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

Build a Graph

In [ ]:
from langgraph.graph import StateGraph, START, END

In [ ]:
builder = StateGraph(MyMessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_edge(START, "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)
graph = builder.compile()

Inference

In [ ]:
from langchain_core.messages import HumanMessage

messages = graph.invoke({"messages": HumanMessage(content="Hello!")})
for m in messages['messages']:
    print(m)

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Multiply 2 and 3")})
for m in messages['messages']:
  print(m)

Dynamic Routing

In [ ]:
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

1 - ToolNode: A prebuilt node provided by LangGraph that executes tools. Think of it as a designated area in our dinner party where a specialized assistant, such as a math expert, works.

2 - tools_condition: A prebuilt condition that examines the AI’s latest response to determine whether it includes a tool call. It is like a decision-maker who looks at the conversation and decides whether to send a request to the math expert or continue the conversation normally. When it detects a tool call, it automatically routes to a node

In [ ]:
# Build graph
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)

# A ToolNode automatically handles executing any tool calls made by the LLM
builder.add_node("tools", ToolNode([multiply]))

# Connect the start to our LLM node
builder.add_edge(START, "tool_calling_llm")

# Add a conditional edge that uses 'tools_condition'
# If the LLM’s response indicates a tool call, it routes to "tools"
# Otherwise, it routes to END
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition,
)

builder.add_edge("tools", END)
graph = builder.compile()

Inference

In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="Multiply 3 and 2")]
messages = graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="Hello world.")]
messages = graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()
